## Lesson 2: Baseline Email Assistant

This lesson builds an email assistant that:

- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [3]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently.",
}

In [4]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@example.com>",
    "to": "John Doe <john.doe@example.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs.
Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

### Define the first part of the agent - triage


In [5]:
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal, TypedDict

In [6]:
from langchain_openai.chat_models import ChatOpenAI

In [7]:
llm = ChatOpenAI(
    # model="openrouter/cypher-alpha:free",
    # base_url="https://openrouter.ai/api/v1",
    model="gpt-4.1",
    base_url="https://models.inference.ai.azure.com/",
    temperature=0.5,
)

In [8]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [9]:
llm_router = llm.with_structured_output(schema=Router)

In [10]:
from prompts import triage_system_prompt, triage_user_prompt

In [11]:
print(triage_system_prompt)


< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few

In [12]:
print(triage_user_prompt)


Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}


In [13]:
system_prompt = triage_system_prompt.format(
    full_name=profile["full_name"],
    name=profile["name"],
    examples=None,
    user_profile_background=profile["user_profile_background"],
    triage_no=prompt_instructions["triage_rules"]["ignore"],
    triage_notify=prompt_instructions["triage_rules"]["notify"],
    triage_email=prompt_instructions["triage_rules"]["respond"],
)

In [14]:
user_prompt = triage_user_prompt.format(
    author=email["from"],
    to=email["to"],
    subject=email["subject"],
    email_thread=email["body"],
)

In [15]:
result = llm_router.invoke(
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
)

In [16]:
print(f"{result.reasoning=}\n\n{result.classification=}")  # type: ignore

result.reasoning="Alice is asking a direct question regarding the API documentation and whether certain endpoints were intentionally omitted or if the documentation needs to be updated. This is a specific inquiry related to John's responsibilities as a senior software engineer leading the team, and it requires his input or clarification."

result.classification='respond'


### Main agent, define tools


In [17]:
from langchain_core.tools import tool

In [18]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""

    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"

In [19]:
@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: str
) -> str:
    """Schedule a calendar meeting."""

    # Placeholder response - in real app would check calendar and schedule
    return (
        f"Meeting '{subject}' which lasts {duration_minutes} mins has been "
        f"scheduled for {preferred_day} with {len(attendees)} attendees"
    )

In [20]:
@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""

    # Placeholder response - in real app would check actual calendar
    return f"Available slots on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

### Main agent, define prompt


In [21]:
from prompts import agent_system_prompt

In [22]:
print(agent_system_prompt)


< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >



In [23]:
def create_prompt(state: dict[str, list]) -> list[dict[str, str]]:
    return [
        {
            "role": "system",
            "content": agent_system_prompt.format(
                instructions=prompt_instructions["agent_instructions"], **profile
            ),
        }
    ] + state["messages"]

In [24]:
from langgraph.prebuilt import create_react_agent

In [25]:
tools: list = [write_email, schedule_meeting, check_calendar_availability]

In [26]:
agent = create_react_agent(model=llm, tools=tools, prompt=create_prompt)

In [27]:
response = agent.invoke(
    input={
        "messages": [
            {
                "role": "user",
                "content": "what is my availability for tuesday?",
            }
        ]
    }
)

In [28]:
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

You have availability on Tuesday at the following times:
- 9:00 AM
- 2:00 PM
- 4:00 PM

Let me know if you’d like to schedule a meeting or block any of these times.


### Create the Overall Agent


In [29]:
from langgraph.graph import add_messages

In [30]:
class State(TypedDict):
    email_input: dict[str, str]
    messages: Annotated[list, add_messages]

In [ ]:
from typing import Literal

from langgraph.graph import END, START, StateGraph
from langgraph.types import Command

In [32]:
def triage_router(state: State) -> Command[Literal["response_agent", "__end__"]]:
    """Analyze the unread email and route it according to its content."""

    author = state["email_input"]["author"]
    to = state["email_input"]["to"]
    subject = state["email_input"]["subject"]
    email_thread = state["email_input"]["email_thread"]

    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        examples=None,
        user_profile_background=profile["user_profile_background"],
        triage_no=prompt_instructions["triage_rules"]["ignore"],
        triage_notify=prompt_instructions["triage_rules"]["notify"],
        triage_email=prompt_instructions["triage_rules"]["respond"],
    )

    user_prompt = triage_user_prompt.format(
        author=author,
        to=to,
        subject=subject,
        email_thread=email_thread,
    )

    result = llm_router.invoke(
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    match result.classification:  # type: ignore
        case "respond":
            print("Classification: Respond - This email requires a response.")

            goto = "response_agent"
            update = {
                "messages": [
                    {
                        "role": "user",
                        "content": f"Respond to the email {state['email_input']}",
                    }
                ]
            }
        case "ignore":
            print(
                "Classification: Ignore - This email is not relevant and can be ignored."
            )

            goto = END
            update = None
        case "notify":
            print("Classification: Notify - This email contains important information.")

            goto = END
            update = None
        case _:
            raise ValueError(f"Unexpected classification: {result.classification}")  # type: ignore

    return Command(goto=goto, update=update)

### Put it all together


In [33]:
email_agent = StateGraph(state_schema=State)

email_agent = email_agent.add_node(node=triage_router)
email_agent = email_agent.add_node(node="response_agent", action=agent)
email_agent = email_agent.add_edge(start_key=START, end_key="triage_router")

email_agent = email_agent.compile()

In [ ]:
email_input = {
    "author": "Marketing Team <marketing@amazingdeals.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "🔥 EXCLUSIVE OFFER: Limited Time Discount on Developer Tools! 🔥",
    "email_thread": """Dear Valued Developer,

Don't miss out on this INCREDIBLE opportunity!

🚀 For a LIMITED TIME ONLY, get 80% OFF on our Premium Developer Suite!

✨ FEATURES:
- Revolutionary AI-powered code completion
- Cloud-based development environment
- 24/7 customer support
- And much more!

💰 Regular Price: $999/month
🎉 YOUR SPECIAL PRICE: Just $199/month!

🕒 Hurry! This offer expires in:
24 HOURS ONLY!

Click here to claim your discount: https://amazingdeals.com/special-offer

Best regards,
Marketing Team
---
To unsubscribe, click here
""",
}

In [35]:
response = email_agent.invoke({"email_input": email_input})

Task triage_router with path ('__pregel_pull', 'triage_router') wrote to unknown channel branch:to:__end__, ignoring it.


Classification: Ignore - This email is not relevant and can be ignored.


In [36]:
email_input = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [37]:
response = email_agent.invoke({"email_input": email_input})

Classification: Respond - This email requires a response.


In [38]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi John,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_Zz1CszuL1ILhndSxq6E9kC5W)
 Call ID: call_Zz1CszuL1ILhndSxq6E9kC5W
  Args:
    to: alice.smith@company.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thanks for bringing this to my attention. The /auth/refresh and /auth/validate endpoints should indeed be included in t